In [2]:
#для VMAE, BLIP, BART
%pip install torch torchvision transformers timm einops opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#для WHISPER
!sudo apt install ffmpeg -y

In [3]:
#для WHISPER
%pip install --upgrade transformers accelerate ffmpeg-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
#для VMAE, BLIP, BART
import torch
import torchvision.transforms as T
import numpy as np
import cv2  # OpenCV для обработки видео
import os
from transformers import VideoMAEModel, VideoMAEFeatureExtractor
from transformers import BlipForConditionalGeneration, BlipProcessor
from transformers import BartForConditionalGeneration, BartTokenizer
from PIL import Image


# Определение устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#для WHISPER
import torch
import pandas as pd
import os
import ffmpeg
from pathlib import Path
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device2 = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device2}")
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

Using device: cuda:0


In [6]:
video_path = "resources/source_video"
audio_path = "resources/converted_audio"

In [18]:
#методы VMAE, BLIP, BART
def load_video_frames_from_mp4(path, max_frames=16, resize=(224, 224)):
    """
    Извлечение кадров из видеофайла и их предобработка.

    Args:
        path (str): Путь к видеофайлу (.mp4).
        max_frames (int): Максимальное количество кадров для обработки.
        resize (tuple): Размер для изменения размера кадров.

    Returns: torch.Tensor: Тензор вида [1, num_frames, 3, height, width]
    """
    frames = []
    cap = cv2.VideoCapture(path)

    if not cap.isOpened():
        raise ValueError(f"Не удалось открыть видеофайл: {path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Общее количество кадров в видео: {total_frames}")

    frame_idx = 0
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            print("Достигнут конец видео или произошла ошибка при чтении кадра.")
            break

        # Конвертация из BGR (OpenCV) в RGB (PIL)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame)

        transform = T.Compose([
            T.Resize(resize),
            T.ToTensor(),
            T.Normalize(mean=[0.5, 0.5, 0.5],  # Средние значения нормализации
                        std=[0.5, 0.5, 0.5])   # Стандартные отклонения нормализации
        ])
        image = transform(image)
        frames.append(image)
        frame_idx += 1

    cap.release()

    if len(frames) == 0:
        raise ValueError("Не удалось извлечь ни одного кадра из видео.")

    # Если кадров меньше, чем max_frames, дублируем последние кадры
    while len(frames) < max_frames:
        frames.append(frames[-1].clone())

    video_tensor = torch.stack(frames)  # [num_frames, 3, height, width]
    video_tensor = video_tensor.unsqueeze(0)  # [1, num_frames, 3, height, width]
    return video_tensor

def extract_features_with_videomae(pixel_values, device):
    """
    Извлечение признаков из видео с использованием предобученной модели VideoMAE.

    Args:
        pixel_values (torch.Tensor): Тензор формы [batch_size, num_frames, 3, height, width]
        device (torch.device): Устройство для вычислений (GPU или CPU)

    Returns:
        torch.Tensor: Извлеченные признаки.
    """
    # Загрузка предобученного извлекателя признаков и модели
    feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-large")
    model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-large")
    model.to(device)
    model.eval()

    # Перемещение данных на устройство
    pixel_values = pixel_values.to(device)

    # Проверка формы входных данных
    if pixel_values.ndim != 5:
        raise ValueError(f"Ожидается 5 измерений, получено {pixel_values.ndim}")

    batch_size, num_frames, num_channels, height, width = pixel_values.shape
    assert num_channels == 3, "Ожидается 3 канала (RGB)"

    with torch.no_grad():
        outputs = model(pixel_values=pixel_values)
        # Получаем последнее скрытое состояние
        features = outputs.last_hidden_state  # Вид: [batch_size, num_frames, hidden_size]
    return features

def generate_description_with_blip(pixel_values, device):
    """
    Генерация описания к видео с использованием модели BLIP.

    Args:
        pixel_values (torch.Tensor): Тензор формы [batch_size, num_frames, 3, height, width]
        device (torch.device): Устройство для вычислений (GPU или CPU)

    Returns:
        str: Сгенерированное описание.
    """
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
    model.to(device)
    model.eval()

    batch_size, num_frames, num_channels, height, width = pixel_values.shape
    descriptions = []

    with torch.no_grad():
        for i in range(num_frames):
            frame = pixel_values[:, i, :, :, :]  # [batch_size, 3, height, width]
            # Перемещение кадра на устройство
            frame = frame.to(device)

            # Преобразуем тензор в список изображений
            images = frame.cpu().numpy()
            images = [Image.fromarray(np.uint8(np.clip(img.transpose(1, 2, 0) * 255, 0, 255))) for img in images]

            inputs = processor(images=images, return_tensors="pt").to(device)
            outputs = model.generate(**inputs)
            captions = processor.batch_decode(outputs, skip_special_tokens=True)
            descriptions.extend(captions)

    # Объединяем описания отдельных кадров
    final_description = " ".join(descriptions)
    return final_description

def summarize_text(text, device, model_name="facebook/bart-large-cnn", max_length=150, min_length=40):
    """
    Суммаризация текста с использованием предобученной модели.

    Args:
        text (str): Текст для суммаризации.
        device (torch.device): Устройство для вычислений (GPU или CPU).
        model_name (str): Название предобученной модели суммаризации.
        max_length (int): Максимальная длина суммаризованного текста.
        min_length (int): Минимальная длина суммаризованного текста.

    Returns:
        str: Суммаризованный текст.
    """
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)
    model.to(device)
    model.eval()

    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    with torch.no_grad():
        summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def get_text_from_video(video_id):
    path = f'{video_path}/{video_id}.mp4' 
     # Шаг 1: Извлечение кадров из видео
    try:
        pixel_values = load_video_frames_from_mp4(path)
        print(f"Форма pixel_values: {pixel_values.shape}")  # Ожидается [1, num_frames, 3, 224, 224]
    except Exception as e:
        print(f"Ошибка при загрузке кадров: {e}")
        return

    # Шаг 2: Извлечение признаков с помощью VideoMAE
    try:
        features = extract_features_with_videomae(pixel_values, device)
        print(f"Форма извлеченных признаков: {features.shape}")  # Пример: [1, num_frames, hidden_size]
    except Exception as e:
        print(f"Ошибка при извлечении признаков с VideoMAE: {e}")
        return

    # Шаг 3: Генерация описания с помощью BLIP
    try:
        description = generate_description_with_blip(pixel_values, device)
        print(description)
    except Exception as e:
        print(f"Ошибка при генерации описания с BLIP: {e}")
        return

    # Шаг 4: Суммаризация полученного описания
    try:
        summary = summarize_text(description, device)
        return summary
    except Exception as e:
        print(f"Ошибка при суммаризации описания: {e}")
        return   

In [8]:
#whisper model
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device2)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device2,

)

In [9]:
#метод конвертации mp4 в mp3 для whisper
def convert_video_to_audio(input_path, output_path):
    try:
        # Проверяем существование входного файла
        input_file = Path(input_path)
        if not input_file.exists():
            raise FileNotFoundError(f"Файл {input_path} не найден")

        # Выполняем преобразование
        (
            ffmpeg.input(input_path)
            .output(output_path, format='mp3')
            .overwrite_output()
            .run(capture_stdout=True, capture_stderr=True)
        )

        # print(f"Преобразование завершено успешно. Результат сохранен как {output_path}")
    except Exception as e:
        print(f"Произошла ошибка при преобразовании: {str(e)}")

In [20]:
#whisper конвертация и расшифровка аудио с сумаризацией
def get_text_from_audio(video_id):
    convert_video_to_audio(f'{video_path}/{video_id}.mp4', f'{audio_path}/{video_id}.mp3')
    text = pipe(f'{audio_path}/{video_id}.mp3', return_timestamps=True)['text']
    return  summarize_text(text, device)

In [12]:
print(get_text_from_video('1'))

Общее количество кадров в видео: 486
Форма pixel_values: torch.Size([1, 16, 3, 224, 224])
Форма извлеченных признаков: torch.Size([1, 1568, 1024])
a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple background a close up of a computer screen with a purple backgroun

In [13]:
print(get_text_from_audio('1'))

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. Thi

summarize:  Раз, два, три, четыре, пять. Сейчас  работает  фигнёй,    “оже”,  “Не  такте,”  ‘’’, “‘‘,’ ’” ‘, ‘,'’  ’, ’.
